<a href="https://colab.research.google.com/github/zackives/upenn-cis5450-hw/blob/main/13_Module_3_Notebook_I_DimReduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statistical Tests

We saw in the previous notebook how the t-test works. Here let's look at comparing distributions of two categorical attributes.

In [ ]:
import pandas as pd

voters_df = pd.DataFrame([{"gender": "male", "republican": 120, "democrat": 90, "independent": 40}, \
                          {"gender": "female", "republican": 110, "democrat": 95, "independent": 45}])

voters_df.set_index("gender", inplace=True)

voters_df


In [ ]:
import scipy.stats as stats

# The "contingency table" shows
result = stats.chi2_contingency(voters_df)

print(f"Statistic: {result[0]}")
print(f"p-value: {result[1]}")

if result[1] < 0.05:
  print("Statistically significant")
else:
  print("Not statistically significant")

# Reducing Dimensionality



## Autograder setup

In [ ]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 99999999 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

In [ ]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

In [ ]:
%set_env HW_ID=cis5450_25f_HW9

In [ ]:
!pip3 install penngrader-client

In [ ]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

## Looking at Glass

In [ ]:
# Glass data from https://archive.ics.uci.edu/ml/machine-learning-databases/glass/

In [ ]:
!wget https://archive.ics.uci.edu/static/public/42/glass+identification.zip

In [ ]:
!unzip glass+identification.zip

In [ ]:
# Load into a dataframe, with the header in row 0
import pandas as pd

glass_df = pd.read_csv('glass.data',header=None,names=['ID','RefractiveIndex','Na','Mg','Al','Si','K','Ca','Ba','Fe','Label'])

glass_df

### Exploratory Data Analysis

Let's do some "EDA" - exploratory data analysis.  Typically that involves getting a sense of the fields, distirbutions, missing values, correlations, and more.

In [ ]:
glass_df.info()

In [ ]:
# What are the data distributions within each column?
glass_df.describe()

Observe the really wide differences between means, value ranges, and more across the different elements and the refractive index.

Any missing values?

In [ ]:
glass_df.isnull().sum()

No -- that's good!

Now let's look at the value distributions relative to each other...

In [ ]:
glass_df.set_index('ID')
glass_types_df = glass_df[['Label']]

# We don't really need these
glass_df = glass_df.drop(columns=['ID', 'Label'])

display(glass_df)
display(glass_types_df)

In [ ]:
# Let's see the popularity of each label...

glass_types_df.value_counts().plot(kind='bar')

Observe that some values, e.g., 4, don't have any instances.

In [ ]:
# A Pairplot of every item vs every other item
import seaborn as sns
import matplotlib.pyplot as plt

sns.pairplot(glass_df)

In [ ]:
corr_matrix = glass_df.corr()

sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')

Observe that some of the features are fairly (anti-)correlated, e.g., Ba and Mg.

## Unsupervised Machine Learning!

We typically set up machine learning problems as follows.

1. Convert from categorical and other values into numeric values
2. Convert from dataframes to arrays
3. Separate out any classes / labels (like glass type)

We will call the *input data*  $X$ and the *labels* $y$.

In [ ]:
# Set up the problem

X = glass_df.to_numpy()
y = glass_types_df.to_numpy()

In [ ]:
X

In [ ]:
import numpy as np

# This is the covariance matrix
np.cov(X)

Let's look at a couple of dimensions within the data...

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

fig1 = glass_df.plot.scatter(x='RefractiveIndex',y='Na')

## Applying PCA

Let's first scale the data (by removing the mean and scaling by unit variance).

In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardizing the features based on unit variance
X = StandardScaler().fit_transform(X)

print (X.shape)
print(X)

Now we'll re-plot our two dimensions

In [ ]:
# Re-plotting now with the mean at the center!
plt.scatter(X[:,0], X[:,1])

## Actually Running PCA

We'll use, for the first time, a standard sckikit-learn 'flow': create a model, `fit` it, and `transform` the data.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
X2 = pca.fit_transform(X)

In [ ]:
# Let's see the components. There are p of them, each with p elements
pca.components_

`X2` is the transformed matrix, in a new subspace -- we called this `X'`.  `pca.components_` is what we called `W`.

In [ ]:
print (X2.shape)
print (X2)

To get this, we can directly compute the original data times the transformation matrix (transposed).

In [ ]:
X @ pca.components_.T

Again, we can invert the process...

In [ ]:
pca.inverse_transform(X2)

In [ ]:
X2 @ pca.components_

In [ ]:
X

## Visualizing the first 2 components (eigenvectors)

In [ ]:
# Visualization code based on
# https://stackoverflow.com/questions/18299523/basic-example-for-pca-with-matplotlib
import numpy as np

# Let's take our first two dimensions, as before
data = X[:, 0:2]

mu = data.mean(axis=0)
data = (data - mu)/data.std(axis=0)
eigenvectors, eigenvalues, V = np.linalg.svd(data.T, full_matrices=False)
projected_data = np.dot(data, eigenvectors)
sigma = projected_data.std(axis=0).mean()

fig, ax = plt.subplots()
ax.scatter(X[:,0], X[:,1])
for axis in eigenvectors:
    start, end = mu, mu + sigma * axis
    ax.annotate(
        '', xy=end, xycoords='data',
        xytext=start, textcoords='data',
        arrowprops=dict(facecolor='red', width=2.0))
ax.set_aspect('equal')
plt.show()

Let's re-plot by rotating along the first two dimensions

In [ ]:
# Here is the transformed data along the first 2 components
plt.scatter(X2[:,0], X2[:,1])

### How Many Components? Principal Components vs Explained Variance

How much does each component explain the variance?  We can look at the `explained_variance_ratio_` to tell...

In [ ]:
np.set_printoptions(suppress=True)
pca.explained_variance_ratio_

In [ ]:
# See how much is contributed by the first few terms
pc_vs_variance = np.cumsum(pca.explained_variance_ratio_)

pc_vs_variance
plt.plot(pc_vs_variance)

... So, the first 6 components (0 through 5, of 9) give 90% explained variance.  Not too bad!

## PCA and Learning a Predictor (Classifier)

From the above, we saw how to do PCA on the overall dataset.  But let's do it more methodically as part of machine learning.  We'll start with separate training and test data.



In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(\
  X, y, test_size=0.20, random_state=42)

# Fit the PCA on the training data
pca = PCA(n_components=6)
pca.fit(X_train)
# and transform it
X_train_2 = pca.transform(X_train)

# Then train a simple linear regression classifier
# (tries to find the best weighted linear combination to
# match the output)
regr = linear_model.LinearRegression()
regr.fit(X_train_2, y_train)

X_train_2

In [ ]:
X_test_2 = pca.transform(X_test)

regr.predict(X_test_2)

regr.score(X_test_2, y_test)

So, 87.4% predictive accuracy on the test set.

How does that compare with working directly on the real data?

In [ ]:
# Train and evaluate over non-dimensionality-reduced data
regr_full_data = linear_model.LinearRegression()
regr_full_data.fit(X_train, y_train)

regr_full_data.predict(X_test)
regr_full_data.score(X_test, y_test)

Actually better!  How can that be? We not only reduced dimensions, but we removed correlation. The `LinearRegression` classifier assumes uncorrelated features.

## PCA on Spark

Thus far we've seen PCA using Scikit-Learn, which is fantastic for mid-sized data sets.

What if we have a really big Spark dataframe with our dataset?


In [ ]:
%set_env SPARK_VERSION=3.5.6

In [ ]:
## Let's install Apache Spark on Colab

!wget -nc https://downloads.apache.org/spark/spark-$SPARK_VERSION/spark-$SPARK_VERSION-bin-hadoop3.tgz
!tar xf spark-$SPARK_VERSION-bin-hadoop3.tgz
!pip install findspark

import os

os.environ["SPARK_HOME"] = "/content/spark-" + os.environ['SPARK_VERSION'] + "-bin-hadoop3"

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SQLContext

spark = SparkSession.builder.appName('Clustering').getOrCreate()

In [ ]:
import pandas as pd
from pyspark import SparkFiles

from pyspark.sql.types import StringType, IntegerType, DoubleType, StructField, StructType, ArrayType, MapType

# ID,RefractiveIndex,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
schema = StructType([
        StructField("ID", IntegerType(), True),
        StructField("RefractiveIndex", DoubleType(), True),
        StructField("Na", DoubleType(), True),
        StructField("Mg", DoubleType(), True),
        StructField("Al", DoubleType(), True),
        StructField("Si", DoubleType(), True),
        StructField("K", DoubleType(), True),
        StructField("Ca", DoubleType(), True),
        StructField("Ba", DoubleType(), True),
        StructField("Fe", DoubleType(), True),
        StructField("Type", IntegerType(), True),
         ])

glass_sdf = spark.createDataFrame(\
                                  pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data'), \
                                  schema=schema)

glass_sdf.show(5)

From Spark, we need to compute  a matrix (specifically, a Row Matrix) for MLLib's linear algebra operators to work on.
Then we can call `computePrincipalComponents`.

In [ ]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

M = RowMatrix(glass_sdf.select('RefractiveIndex','Na','Mg','Al','Si','K','Ca','Ba','Fe').rdd.map(\
  lambda row: Vectors.dense(list(row.asDict().values()))))

pc = M.computePrincipalComponents(6)

projected = M.multiply(pc)

projected.rows.collect()

## t-SNE

For high-dimensional data, we often use t-Distributed Stochastic Neighbor Embedding (t-SNE) to reduce dimensionality.  This is a stochastic method so it doesn't always produce the same output.

t-SNE isn't supported directly in Apache Spark (there is a 3rd party extension) but it's built into SciKit-Learn.

In [ ]:
from sklearn.manifold import TSNE

X_embedded = TSNE(n_components=2).fit_transform(X)
plt.scatter(X_embedded[:,0],X_embedded[:,1])

## Exercise

Let's take another popular dataset, of housing prices in Boston.

In [ ]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

### Scaling

Don't forget to rescale your data

In [ ]:
from sklearn.preprocessing import StandardScaler

# TODO

Plot a heatmap to see if there is a lot of correlation.

In [ ]:
# TODO: compute correlation matrix, plot heatmap

In [ ]:
# We'll fit a PCA model without reducing any
# dimensions here
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X)

Now plot the explained variance and find the total number of dimensions that will get us to 95% or higher. Recall that this will count Dimension 0 so your count will be 1 more than the last component.

In [ ]:
# TODO: explained variance ratio curve

In [ ]:
# How many dimensions should we use, to get 95% explained variance ratio?
dimensions = # TODO for the number of component dimensions
dimensions

In [ ]:
grader.grade('pca', dimensions)